In [2]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import nn
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
import joblib
import biGRU_model 
import gbm_model
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
import pandas as pd
import numpy as np
from pandarallel import pandarallel

pandarallel.initialize()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
gbm = CatBoostClassifier()      
gbm.load_model('models/catboost_1')
vocab = SimpleVocabulary(save_path="./models/vocab.dict")
gru = torch.load('models/biGRU')
device = torch.device('cpu') 
gru = biGRU_model.BiGRU(vocab.count, embedding_dim=10, hidden_size=50, device='cpu') 
gru.load_state_dict(torch.load('models/biGRU', map_location=device))

#tfidf = tfidf = joblib.load('models/tfidf.pkl') 
#svdt = tfidf = joblib.load('models/svdt.pkl') 


2020-09-26 23:35:42.659 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead
2020-09-26 23:35:42.660 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /rapids/notebooks/my_data/BMSTU_hack/models/vocab.dict]


<All keys matched successfully>

In [25]:
data = pd.read_csv('data/level0.csv')
labels = data['label']
data.drop(['label'], axis=1, inplace=True)
data_gbm = gbm_model.add_features(data.copy(), is_fitted=True)
data_rnn = data.to_numpy()

loading existing models...


In [26]:
torch.cuda.empty_cache()
gbm_proba = gbm.predict_proba(data_gbm.to_numpy())[:,1].flatten()
rnn_proba = biGRU_model.predict_proba(gru, vocab, data_rnn, device='cpu').flatten()
X = np.column_stack((gbm_proba, rnn_proba))

In [ ]:
clf = LogisticRegression(random_state=0).fit(X, labels)

In [6]:
print(np.mean(abs(gbm_proba - rnn_proba)), np.std(abs(gbm_proba - rnn_proba)))

0.10794858100071444 0.162942920578531


In [17]:
import seaborn as sns

In [9]:
biGRU_model.predict_proba(gru, vocab, np.array([['theopportunitycollection.com']]), device='cpu')

array([0.58014107])